In [ ]:
import os
import json
import asyncio
import faiss
import numpy as np
import pickle
import pandas as pd
import re
import unicodedata
from bs4 import BeautifulSoup
from stopwordsiso import stopwords
from telethon import TelegramClient
import nest_asyncio
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
from sentence_transformers import SentenceTransformer

# Путь к JSON-файлу с данными новостей (при необходимости измените)
JSON_PATH = "news_data2.json"

# Директория для сохранения изображений (создаётся автоматически, если не существует)
SAVE_DIR = "/Applications/Study/Diploma/images"
os.makedirs(SAVE_DIR, exist_ok=True)

# Параметры доступа к API Telegram
api_id = 25033293        # API ID вашего приложения
api_hash = "7af4a394d50e40a4b7475d5c87d110dd"  # API Hash вашего приложения

# Список каналов, сгруппированных по категориям
channels = [
    ["@rian_ru"],
    ["@sportrian", "@sportsru", "@UFCRussia", "@football_nik", "@hockey_vbros"],
    ["@rianovostiAmerica", "@politica_media", "@vv_volodin", "@margaritasimonyan"],
    ["@World_Sanctions"],
    ["@igmtv", "@inQsitor"],
    ["@movie7channel", "@marvel4"],
    ["@rasofficial"]
]

# Соответствующие им названия категорий
categories = ["General news", "Sport", "Politic", "Economic", "Games", "Films and serials", "Science"]

# Карта канала → категория для быстрого присвоения
channel_to_category = {}
for cat, ch_list in zip(categories, channels):
    for ch in ch_list:
        channel_to_category[ch] = cat

# Загрузка моделей эмбеддингов (SentenceTransformer, CLIP, BLIP)
print("Загрузка моделей эмбеддингов (SentenceTransformer, CLIP, BLIP)...")
text_encoder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
clip_model     = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model     = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
print("Модели загружены. Запуск цикла обновления...")

# Пути к файлам FAISS-индексов и вспомогательным данным
TEXT_INDEX_PATH    = "news_date.faiss"        # Индекс для текстовых эмбеддингов
CLIP_INDEX_PATH    = "clip_index_last.faiss"  # Индекс для эмбеддингов изображений (CLIP)
BLIP_INDEX_PATH    = "blip_index_last.faiss"  # Индекс для эмбеддингов подписей BLIP
CLIP_PATHS_PATH    = "clip_valid_paths.pkl"   # Сохранённые пути к изображениям (CLIP)
BLIP_CAPTIONS_PATH = "blip_captions.pkl"      # Сохранённые подписи (BLIP)

# Асинхронный основной цикл: соединение, загрузка, проверка и обновление индексов
async def update_loop():
    # Запуск и подключение клиента Telegram
    client = TelegramClient('session_name', api_id, api_hash)
    await client.start()
    print("Клиент Telegram запущен и подключен.")

    # Загрузка существующих данных или инициализация пустых структур
    if os.path.exists(JSON_PATH):
        with open(JSON_PATH, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
        # Проверим, что все ключи есть даже в пустом файле
        for key in ["category", "channel", "message_id", "time", "text", "image_path"]:
            existing_data.setdefault(key, [])
    else:
        existing_data = {"category": [], "channel": [], "message_id": [], "time": [], "text": [], "image_path": []}

    # Определение последнего обработанного ID для каждого канала
    last_ids = {}
    if existing_data["channel"]:
        for ch, msg_id in zip(existing_data["channel"], existing_data["message_id"]):
            last_ids[ch] = max(last_ids.get(ch, 0), msg_id)
    else:
        # Если данных нет, установим 0, чтобы получить начальную выборку
        last_ids = {ch: 0 for ch_list in channels for ch in ch_list}

    # Если JSON был пуст — первичная загрузка по 400 сообщений из каждого канала
    if not existing_data["message_id"]:
        print("Первичная загрузка данных из всех каналов...")
        for cat, ch_list in zip(categories, channels):
            for ch_name in ch_list:
                try:
                    entity = await client.get_entity(ch_name)
                    msgs = await client.get_messages(entity, limit=400)
                except Exception as e:
                    print(f"Ошибка при загрузке initial-сообщений из {ch_name}: {e}")
                    continue
                for msg in msgs:
                    existing_data["category"].append(cat)
                    existing_data["channel"].append(ch_name)
                    existing_data["message_id"].append(msg.id)
                    existing_data["time"].append(msg.date.strftime("%d.%m.%Y"))
                    existing_data["text"].append(msg.message)
                    if msg.photo:
                        path = os.path.join(SAVE_DIR, f"{ch_name[1:]}_{msg.id}.jpg")
                        try:
                            await msg.download_media(file=path)
                        except Exception as e:
                            print(f"Не удалось скачать изображение {path}: {e}")
                            path = ""
                        existing_data["image_path"].append(path)
                    else:
                        existing_data["image_path"].append("")
                if msgs:
                    last_ids[ch_name] = max(m.id for m in msgs)
        # Сохраняем начальные данные
        with open(JSON_PATH, 'w', encoding='utf-8') as f:
            json.dump(existing_data, f, ensure_ascii=False, indent=4)
        print(f"Сохранено {len(existing_data['message_id'])} записей в {JSON_PATH}.")

    # Кэш объектов каналов, чтобы не делать повторные запросы
    channel_entities = {}
    iteration = 0

    # Вечный цикл: проверяем новые сообщения, обновляем JSON и FAISS-индексы
    while True:
        # Загрузим текущие индексы
        text_index = faiss.read_index(TEXT_INDEX_PATH)
        clip_index = faiss.read_index(CLIP_INDEX_PATH)
        blip_index = faiss.read_index(BLIP_INDEX_PATH)

        # Собираем новые записи
        new_entries = {"category": [], "channel": [], "message_id": [], "time": [], "text": [], "image_path": []}
        for cat, ch_list in zip(categories, channels):
            for ch_name in ch_list:
                if ch_name not in channel_entities:
                    try:
                        channel_entities[ch_name] = await client.get_entity(ch_name)
                    except Exception as e:
                        print(f"Не удалось получить сущность {ch_name}: {e}")
                        continue
                entity = channel_entities[ch_name]
                last_known = last_ids.get(ch_name, 0)
                try:
                    if last_known > 0:
                        msgs = await client.get_messages(entity, min_id=last_known, limit=None)
                    else:
                        msgs = await client.get_messages(entity, limit=100)
                except Exception as e:
                    print(f"Ошибка при получении сообщений из {ch_name}: {e}")
                    continue
                msgs = sorted(msgs, key=lambda m: m.id)
                for msg in msgs:
                    if last_known and msg.id <= last_known:
                        continue
                    new_entries["category"].append(cat)
                    new_entries["channel"].append(ch_name)
                    new_entries["message_id"].append(msg.id)
                    new_entries["time"].append(msg.date.strftime("%d.%m.%Y"))
                    new_entries["text"].append(msg.message)
                    if msg.photo:
                        path = os.path.join(SAVE_DIR, f"{ch_name[1:]}_{msg.id}.jpg")
                        try:
                            await msg.download_media(file=path)
                        except Exception as e:
                            print(f"Не удалось скачать новое изображение {path}: {e}")
                            path = ""
                        new_entries["image_path"].append(path)
                    else:
                        new_entries["image_path"].append("")
                    last_ids[ch_name] = max(last_known, msg.id)

        # Если пришли новые данные, добавляем их в JSON и обновляем индексы
        if new_entries["message_id"]:
            for k in new_entries:
                existing_data[k].extend(new_entries[k])
            with open(JSON_PATH, 'w', encoding='utf-8') as f:
                json.dump(existing_data, f, ensure_ascii=False, indent=4)
            print(f"Добавлено {len(new_entries['message_id'])} новых записей, всего теперь {len(existing_data['message_id'])}.")

            # Циклически обновляем по одному индексу за проход
            for idx in range(3):
                if idx == 0:
                    # Текстовый индекс
                    print("Обновление текстового FAISS-индекса...")
                    df = pd.DataFrame(new_entries)
                    df["text"] = df["text"].fillna("").astype(str)
                    df = df[df["text"].str.strip().ne("")].reset_index(drop=True)
                    ru_stop = stopwords("ru")
                    def clean_text(t: str) -> str:
                        txt = BeautifulSoup(t, "html.parser").get_text(" ", strip=True)
                        txt = unicodedata.normalize("NFKC", txt).lower()
                        txt = re.sub(r"[^\w\s]", " ", txt)
                        return " ".join([w for w in txt.split() if w not in ru_stop])
                    df["clean_text"] = df["text"].map(clean_text)
                    df["wc"] = df["clean_text"].str.split().apply(len)
                    df = df[df["wc"] > 7].reset_index(drop=True)
                    df.drop(columns=["wc"], inplace=True)
                    months = {
                        '01': 'января','02': 'февраля','03': 'марта','04': 'апреля',
                        '05': 'мая','06': 'июня','07': 'июля','08': 'августа',
                        '09': 'сентября','10': 'октября','11': 'ноября','12': 'декабря'
                    }
                    df["date_ru"] = df["time"].str.replace(
                        r"(\\d{2})\\.(\\d{2})\\.\\d{4}",
                        lambda m: f"{int(m.group(1))} {months[m.group(2)]}",
                        regex=True
                    )
                    texts = (df["date_ru"] + " " + df["clean_text"]).tolist()
                    emb = text_encoder.encode(texts, show_progress_bar=False)
                    vecs = np.array(emb, dtype="float32")
                    faiss.normalize_L2(vecs)
                    text_index.add(vecs)
                    faiss.write_index(text_index, TEXT_INDEX_PATH)
                    print(f"Текстовый индекс: {text_index.ntotal} векторов.")
                elif idx == 1:
                    # CLIP-индекс (изображения)
                    print("Обновление CLIP-индекса изображений...")
                    paths = [p for p in new_entries["image_path"] if p]
                    embs, valid = [], []
                    for p in paths:
                        try:
                            img = Image.open(p).convert("RGB")
                            inp = clip_processor(images=img, return_tensors="pt")
                            with torch.no_grad():
                                im_emb = clip_model.get_image_features(**inp)
                            arr = im_emb.cpu().numpy().astype("float32")[0]
                            embs.append(arr)
                            valid.append(p)
                        except Exception as e:
                            print(f"Ошибка CLIP для {p}: {e}")
                    if embs:
                        arr = np.vstack(embs)
                        clip_index.add(arr)
                        faiss.write_index(clip_index, CLIP_INDEX_PATH)
                        with open(CLIP_PATHS_PATH, "wb") as f:
                            pickle.dump(valid, f)
                        print(f"CLIP-индекс: {clip_index.ntotal} изображений.")
                else:
                    # BLIP-индекс (автоматические подписи)
                    print("Обновление BLIP-индекса подписей...")
                    paths = [p for p in new_entries["image_path"] if p]
                    caps, valid = [], []
                    for p in paths:
                        try:
                            img = Image.open(p).convert("RGB")
                            inp = blip_processor(images=img, return_tensors="pt")
                            with torch.no_grad():
                                outs = blip_model.generate(**inp)
                            cap = blip_processor.batch_decode(outs, skip_special_tokens=True)[0].strip()
                            caps.append(cap)
                            valid.append(p)
                        except Exception as e:
                            print(f"Ошибка BLIP для {p}: {e}")
                    if caps:
                        emb_caps = text_encoder.encode(caps, show_progress_bar=False)
                        arr = np.array(emb_caps, dtype="float32")
                        faiss.normalize_L2(arr)
                        blip_index.add(arr)
                        faiss.write_index(blip_index, BLIP_INDEX_PATH)
                        with open(BLIP_CAPTIONS_PATH, "wb") as f:
                            pickle.dump(caps, f)
                        print(f"BLIP-индекс: {blip_index.ntotal} подписей.")
            print(f"Итерация {iteration} завершена.\n{'-'*60}")
            iteration += 1

        # Пауза перед следующей проверкой
        await asyncio.sleep(300)

# Запуск
if __name__ == "__main__":
    nest_asyncio.apply()
    try:
        asyncio.run(update_loop())
    except KeyboardInterrupt:
        print("Автоматизация остановлена пользователем.")